<a href="https://colab.research.google.com/github/seismosmsr/machine_learning/blob/main/conditional_gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conditional GAN
**Author:** [Aron Boettcher](spectral.online)<br>
**Adapted from:** [Sayak Paul](https://twitter.com/RisingSayak)<br>
**Date created:** 2021/07/13<br>
**Last modified:** 2021/07/15<br>
**Description:** Training a GAN conditioned on class labels to generate handwritten digits.

Generative Adversarial Networks (GANs) let us generate novel image data, video data,
or audio data from a random input. Typically, the random input is sampled
from a normal distribution, before going through a series of transformations that turn
it into something plausible (image, video, audio, etc.).

However, a simple [DCGAN](https://arxiv.org/abs/1511.06434) doesn't let us control
the appearance (e.g. class) of the samples we're generating. For instance,
with a GAN that generates MNIST handwritten digits, a simple DCGAN wouldn't let us
choose the class of digits we're generating.
To be able to control what we generate, we need to _condition_ the GAN output
on a semantic input, such as the class of an image.

In this example, we'll build a **Conditional GAN** that can generate MNIST handwritten
digits conditioned on a given class. Such a model can have various useful applications:

* let's say you are dealing with an
[imbalanced image dataset](https://developers.google.com/machine-learning/data-prep/construct/sampling-splitting/imbalanced-data),
and you'd like to gather more examples for the skewed class to balance the dataset.
Data collection can be a costly process on its own. You could instead train a Conditional GAN and use
it to generate novel images for the class that needs balancing.
* Since the generator learns to associate the generated samples with the class labels,
its representations can also be used for [other downstream tasks](https://arxiv.org/abs/1809.11096).

Following are the references used for developing this example:

* [Conditional Generative Adversarial Nets](https://arxiv.org/abs/1411.1784)
* [Lecture on Conditional Generation from Coursera](https://www.coursera.org/lecture/build-basic-generative-adversarial-networks-gans/conditional-generation-inputs-2OPrG)

If you need a refresher on GANs, you can refer to the "Generative adversarial networks"
section of
[this resource](https://livebook.manning.com/book/deep-learning-with-python-second-edition/chapter-12/r-3/232).

This example requires TensorFlow 2.5 or higher, as well as TensorFlow Docs, which can be
installed using the following command:

In [1]:
!pip install -q git+https://github.com/tensorflow/docs

## Imports

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow_docs.vis import embed
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import imageio

## Constants and hyperparameters

In [3]:
batch_size = 256
num_channels = 1
num_classes = 101
image_size = 512
latent_dim = 512

## Loading the MNIST dataset and preprocessing it

In [4]:
!pip install gdown
import gdown
import zipfile
import os

# I switched to pngs and jpgs to try and use tensorflows native vectorization
#todo: get gdal working so you can just use geotiff
url = 'https://drive.google.com/uc?id=1kwU_CkP1X5M1CDlYKPkb9p5J8GulWBXr'
output = 'sample_test.csv'

#Update with proper validation data
# https://drive.google.com/file/d/1unLB1XCJHoul3gqGYGzqSS8QMr5ZvJ9h/view?usp=sharing
gdown.download(url,output,quiet = False)

url = 'https://drive.google.com/uc?id=1graojPHaMvZGHmocsUFLds9bFWQG_mr6'
output = 'sample_train.csv'

#Update with proper validation data
# https://drive.google.com/file/d/1unLB1XCJHoul3gqGYGzqSS8QMr5ZvJ9h/view?usp=sharing
gdown.download(url,output,quiet = False)
# cwd = os.getcwd()
# with zipfile.ZipFile(cwd+'/sample.zip', 'r') as zip_ref:
#     zip_ref.extractall(cwd+'/sample')

# PATH = os.path.join(os.path.dirname(cwd+'/sample/'), 'sample/')
# print(PATH)

Downloading...
From: https://drive.google.com/uc?id=1kwU_CkP1X5M1CDlYKPkb9p5J8GulWBXr
To: /content/sample_test.csv
100%|██████████| 137M/137M [00:00<00:00, 146MB/s]
Downloading...
From: https://drive.google.com/uc?id=1graojPHaMvZGHmocsUFLds9bFWQG_mr6
To: /content/sample_train.csv
100%|██████████| 13.8G/13.8G [01:36<00:00, 143MB/s]


'sample_train.csv'

In [5]:
# generate n real samples with class labels
def generate_real_samples(s,filename = '/content/sample.csv'):
  import random
  import pandas
  import numpy
	# Sample s rows of data.frame

  #number of records in file (excludes header)
  n = sum(1 for line in open(filename)) - 1 

  #the 0-indexed header will not be included in the skip list
  skip = sorted(random.sample(range(1,n+1),n-s)) 
  df = pandas.read_csv(filename, skiprows=skip)
  X = []
  for i in range(len(df.index)):
    text_exa = df['rh'][i]
    text_exa = str(text_exa).replace("{","").replace("}", "")
    test_exa = text_exa.split(",")
    test_exa = [float(i) for i in test_exa]
    # test_exa = test_exa
    X.append(test_exa)


	# generate class labels
  y = []
  for i in range(len(df.index)):
    # y_one = numpy.ones(1)
    y_one = df['ls'][i]
    y_one = str(y_one).replace("{","").replace("}", "")
    y_one = y_one.split(",")
    y_one = [float(i) for i in y_one]
    y.append(y_one)

    
  X = numpy.array(X)
  y = numpy.array(y)

  # X = (X.astype("float32") / 65455.0).astype("float32")
  # y =(y.astype("float32")+100)/255
  X = ((X.astype("float32")+100/255)).astype("float32")
  y =(y.astype("float32")/ 65455.0).astype("float32")
  return X, y

In [6]:
# generate_real_samples(2)

In [7]:
import numpy as np
import tensorflow as tf

# We'll use all the available examples from both the training and test
# sets.
# (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
(x_train, y_train) = generate_real_samples(450000,filename = '/content/sample_train.csv')
(x_test, y_test) = generate_real_samples(2000,filename = '/content/sample_test.csv')

# all_pixels = x_train
# all_labels = np.concatenate([y_train, y_test])

# # Scale based on reflectence values:
# # https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2?hl=en#bands
# x_train = (x_train.astype("float32") / 65455.0).astype("float32")
# all_labels =(y_train.astype("float32")+100)/255

# x_test = (x_test.astype("float32") / 65455.0).astype("float32")
# y_test =(y_test.astype("float32")+100)/255
# # Create tf.data.Dataset.
dataset_train = tf.data.Dataset.from_tensor_slices((y_train,x_train))
dataset_train = dataset_train.shuffle(buffer_size=1024).batch(batch_size)

dataset_test = tf.data.Dataset.from_tensor_slices((y_test,x_test))
dataset_test = dataset_test.shuffle(buffer_size=1024).batch(batch_size)

# print(f"Shape of training images: {all_labels[1]}")
# print(f"Shape of training images: {all_pixels.shape}")
# print(f"Shape of training labels: {all_labels.shape}")

In [9]:
# print(x_train[1])

## Calculating the number of input channel for the generator and discriminator

In a regular (unconditional) GAN, we start by sampling noise (of some fixed
dimension) from a normal distribution. In our case, we also need to account
for the class labels. We will have to add the number of classes to
the input channels of the generator (noise input) as well as the discriminator
(generated image input).

In [10]:
generator_in_channels = latent_dim + image_size
discriminator_in_channels = image_size + num_classes
print(generator_in_channels, discriminator_in_channels)

1024 613


## Creating the discriminator and generator

The model definitions (`discriminator`, `generator`, and `ConditionalGAN`) have been
adapted from [this example](https://keras.io/guides/customizing_what_happens_in_fit/).

In [11]:
from tensorflow.python.ops.gen_math_ops import sigmoid
from keras import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Concatenate
from keras import Model

# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((discriminator_in_channels,)),
     
        layers.Dense(1024, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Dense(512, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Dense(512, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Flatten(),
        layers.Dense(1024, activation='relu'),
     
        layers.Dense(1,activation = 'sigmoid'),
    ],
    name="discriminator",)

# Create the generator.
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
     
        layers.Dense(1024, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Dense(512, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Dense(512, activation='relu'),
        layers.Reshape(target_shape=(2,512)),
        layers.Dropout(.25),
        layers.LSTM(32, activation="relu",return_sequences=True),
     
        layers.Flatten(),
        layers.Dense(512, activation='relu'),

        layers.Dense(num_classes,  activation="linear"),
    ],
    name="generator",)

## Creating a `ConditionalGAN` model

In [12]:

class ConditionalGAN(keras.Model):
  
    def __init__(self, discriminator, generator, latent_dim):
        super(ConditionalGAN, self).__init__()
        # Create the discriminator.
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(ConditionalGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, data):
        # Unpack the data.
        real_pixels, real_labels = data
        # print(real_pixels[0])
        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_pixels)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        # print(random_latent_vectors[0])
        random_vector_pixels = tf.concat(
            [random_latent_vectors, real_pixels], axis=1
        )
        # print(random_vector_pixels[0])
        # Decode the noise (guided by labels) to fake images.
        generated_labels = self.generator(random_vector_pixels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_pixel_and_labels = tf.concat([generated_labels, real_pixels], -1)
        real_pixel_and_labels = tf.concat([real_labels, real_pixels], -1)
        combined_images = tf.concat(
            [fake_pixel_and_labels, real_pixel_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, real_pixels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_labels = self.generator(random_vector_labels)
            fake_pixels_and_labels = tf.concat([real_pixels, fake_labels], -1)
            predictions = self.discriminator(fake_pixels_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }


## Training the Conditional GAN

In [13]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0003),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True),
)

In [ ]:
for i in range(5):
  (x_train, y_train) = generate_real_samples(250000,filename = '/content/sample_train.csv')
  (x_test, y_test) = generate_real_samples(20000,filename = '/content/sample_test.csv')
  dataset_train = tf.data.Dataset.from_tensor_slices((y_train,x_train))
  dataset_train = dataset_train.shuffle(buffer_size=1024).batch(batch_size)

  dataset_test = tf.data.Dataset.from_tensor_slices((y_test,x_test))
  dataset_test = dataset_test.shuffle(buffer_size=1024).batch(batch_size)
  
  
  cond_gan.fit(dataset_train, epochs=1)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


977/977 [==============================] - 56s 44ms/step - g_loss: 0.4644 - d_loss: 0.0698


## Interpolating between classes with the trained generator

In [ ]:
from keras import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Concatenate
from keras import Model

input_layer = Input(shape=(512))

dense1 = Dense(1024, activation='relu')(input_layer)
resh1 = Reshape(target_shape=(2,512))(dense1)
dropout1 =  Dropout(.25)(resh1)
lstm1 = LSTM(32, activation="relu",return_sequences=True)(dropout1)
# flat1 = Flatten()(lstm1)
# flat2 = Flatten()(resh1)
# conc1 = Concatenate(axis=1)([flat1,flat2])

dense2 = Dense(512, activation='relu')(lstm1)
resh2 = Reshape(target_shape=(2,512))(dense2)
dropout2 =  Dropout(.25)(resh2)
lstm2 = LSTM(32, activation="relu",return_sequences=True)(dropout2)
# flat3 = Flatten()(lstm2)
# flat4 = Flatten()(resh2)
# conc2 = Concatenate(axis=1)([flat3,flat4])

dense3 = Dense(512, activation='relu')(lstm2)
resh3 = Reshape(target_shape=(2,512))(dense3)
dropout3 =  Dropout(.25)(resh3)
lstm3 = LSTM(32, activation="relu",return_sequences=True)(dropout3)
# flat5 = Flatten()(lstm3)
# flat6 = Flatten()(resh3)
# conc3 = Concatenate(axis=1)([flat5,flat6])
dense4 = Dense(512, activation='relu')(lstm3)
flat6 = Flatten()(dense4)

dense4 = Dense(1024, activation='relu')(flat6)

output_layer = Dense(101, activation='linear')(dense4)
model = Model(inputs=input_layer, outputs=output_layer)
optz = keras.optimizers.Adam(lr=0.0001)
model.compile(loss="MeanSquaredError", optimizer=optz,metrics=['accuracy'])

In [ ]:
from keras import Input
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Dropout

from keras import Model
input_layer = Input(shape=(512))
dense1 = Dense(1024, activation='relu')(input_layer)
dense2 = Dense(1024, activation='relu')(dense1)
dropout1 =  Dropout(.25)(dense2)
resh1 = Reshape(target_shape=(1,16,64))(dropout1)
conv1 = Conv1D(filters=32,
               kernel_size=16,
               strides=1,
               activation='relu')(resh1)
resh1 = Reshape(target_shape=(32,1))(conv1)
pool1 = MaxPooling1D(pool_size=2)(resh1)
lstm1 = LSTM(32, activation="relu")(pool1)
output_layer = Dense(101, activation='linear')(lstm1)
model = Model(inputs=input_layer, outputs=output_layer)
optimizer = keras.optimizers.Adam(lr=0.0001)
model.compile(loss="MeanSquaredError", optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
for i in range(3):
  (x_train, y_train) = generate_real_samples(500000,filename = '/content/sample_train.csv')
  (x_test, y_test) = generate_real_samples(20000,filename = '/content/sample_test.csv')


  dataset_train = tf.data.Dataset.from_tensor_slices((y_train,x_train))
  dataset_train = dataset_train.shuffle(buffer_size=1024).batch(batch_size)

  dataset_test = tf.data.Dataset.from_tensor_slices((y_test,x_test))
  dataset_test = dataset_test.shuffle(buffer_size=1024).batch(batch_size)

  model.fit(dataset_train,validation_data =dataset_test , epochs=10)

In [ ]:
# # import numpy as np
# # import tensorflow as tf

# (x_train, y_train) = generate_real_samples(300000,filename = '/content/sample_train.csv')
# (x_test, y_test) = generate_real_samples(2000,filename = '/content/sample_test.csv')


# dataset_train = tf.data.Dataset.from_tensor_slices((y_train,x_train))
# dataset_train = dataset_train.shuffle(buffer_size=1024).batch(batch_size)

# dataset_test = tf.data.Dataset.from_tensor_slices((y_test,x_test))
# dataset_test = dataset_test.shuffle(buffer_size=1024).batch(batch_size)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

(real_labels, real_pixels) = generate_real_samples(batch_size,filename = '/content/sample_test.csv')
p_test = model.predict(real_pixels)


err_vec = (p_test-real_labels)

mean_err_vec = (np.mean(err_vec,axis=0))
mean_label_vec = (np.mean(real_labels,axis=0))
mean_pred_vec = (np.mean(p_test,axis=0))
fig, ax = plt.subplots()
m = 21
for i in range(batch_size):
  ax.plot(p_test[i],real_labels[i],label=i)

# ax.plot(mean_label_vec,mean_err_vec,color='black')
ax.plot(mean_pred_vec,mean_label_vec,color='black')
# plt.plot([-0.02, 0.07], [-0.02, 0.07], 'k-', lw=2)
ax.set(xlabel='predicted waveform', ylabel='truth waveform',
       title='About as simple as it gets, folks')
ax.grid()

# fig.savefig("test.png")
plt.show()
# print(np.mean(np.mean(p_test,axis=1)))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
m = 21
for i in range(64):
  ax.plot(p_test[i]*255, real_labels[i]*255,label=i)

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()

# fig.savefig("test.png")
plt.show()

In [ ]:
# We first extract the trained generator from our Conditiona GAN.
trained_gen = cond_gan.generator
(real_labels, real_pixels) = generate_real_samples(batch_size,filename = '/content/sample_test.csv')

# batch_size = tf.shape(real_pixels)[0]
random_latent_vectors = tf.random.normal(shape=(batch_size, latent_dim))
# print(random_latent_vectors[0])
random_vector_pixels = tf.concat(
    [random_latent_vectors, real_pixels], axis=1
)
# print(real_labels[1])

generated_waveform = trained_gen(random_vector_pixels)

# print(generated_waveform[1])
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots()
ax.plot(generated_waveform[1], real_labels[1])

ax.set(xlabel='time (s)', ylabel='voltage (mV)',
       title='About as simple as it gets, folks')
ax.grid()

# fig.savefig("test.png")
plt.show()

Here, we first sample noise from a normal distribution and then we repeat that for
`num_interpolation` times and reshape the result accordingly.
We then distribute it uniformly for `num_interpolation`
with the label indentities being present in some proportion.

In [ ]:
fake_images *= 255.0
converted_images = fake_images.astype(np.uint8)
converted_images = tf.image.resize(converted_images, (96, 96)).numpy().astype(np.uint8)
imageio.mimsave("animation.gif", converted_images, fps=1)
embed.embed_file("animation.gif")

We can further improve the performance of this model with recipes like
[WGAN-GP](https://keras.io/examples/generative/wgan_gp).
Conditional generation is also widely used in many modern image generation architectures like
[VQ-GANs](https://arxiv.org/abs/2012.09841), [DALL-E](https://openai.com/blog/dall-e/),
etc.

You can use the trained model hosted on [Hugging Face Hub](https://huggingface.co/keras-io/conditional-gan) and try the demo on [Hugging Face Spaces](https://huggingface.co/spaces/keras-io/conditional-GAN).